test to create a pandas from nodes

In [1]:
import pandas as pd
import numpy as np

In [2]:
# ----------------------------------------------------------------
# Author: Alvaro Paricio Garcia, alvaro.paricio@uah.es
# TAZ CALCULATION BASED ON THE RAY CASTING ALGORYTHM
# FROM : http://rosettacode.org/wiki/Ray-casting_algorithm#Python
# ----------------------------------------------------------------
from collections import namedtuple
from collections import defaultdict
from pprint import pprint as pp
import sys
import os.path
from lxml import etree
import pandas as pd

TazPt      = namedtuple('TazPt', 'x, y')               # Point
TazEdge    = namedtuple('TazEdge', 'a, b')           # Taz edge from a to b
TazPolygon = namedtuple('TazPolygon', 'name, edges')    # TazPolygon

_eps = 0.00001
_huge = sys.float_info.max
_tiny = sys.float_info.min

# ===================================================================
def error( code, text ):
   print("ERROR: "+text)
   sys.exit(code)

# ===================================================================
# PART 1: ALGORITHM
# ===================================================================
def rayintersectseg(p, edge):
    ''' takes a point p=TazPt() and an edge of two endpoints a,b=TazPt() of a line segment returns boolean
    '''
    a,b = edge
    if a.y > b.y:
        a,b = b,a
    if p.y == a.y or p.y == b.y:
        p = TazPt(p.x, p.y + _eps)

    intersect = False

    if (p.y > b.y or p.y < a.y) or (
        p.x > max(a.x, b.x)):
        return False

    if p.x < min(a.x, b.x):
        intersect = True
    else:
        if abs(a.x - b.x) > _tiny:
            m_red = (b.y - a.y) / float(b.x - a.x)
        else:
            m_red = _huge
        if abs(a.x - p.x) > _tiny:
            m_blue = (p.y - a.y) / float(p.x - a.x)
        else:
            m_blue = _huge
        intersect = m_blue >= m_red
    return intersect

def _odd(x): return x%2 == 1

def taz_contains_pt(tazpoly, pt):
    ln = len(tazpoly)
    return _odd(sum(rayintersectseg(pt, edge)
                    for edge in tazpoly.edges ))

def taz_pp(tazpoly):
    print ("\n  TazPolygon(name='%s', edges=(" % tazpoly.name)
    print ('   ', ',\n    '.join(str(e) for e in tazpoly.edges) + '\n    ))')

In [27]:
class MuTazCalculator:

  # -----------------------------------------------
  def __init__(self,opts):
    self.opts = opts
    self.nodes = []
    self.edges = {}
    self.net = []
    self.mutazs = []
    self.flag_create_dataframes = False
    self.geometry = {}
    self.geo_nodes = {}
    self.verbose = opts['verbose']

  # -----------------------------------------------
  def vprint(self,txt):
    if( self.verbose ):
      print( txt )

  # -----------------------------------------------
  def loadNodes(self,file):
    self.vprint( "Start parsing nodes file" );

    root = {}
    n=0
    try:
      tree=etree.parse(file)
      root=tree.getroot()
    except:
      error(2,"Parsing "+file+" file. Exception: "+str(sys.exc_info()) )

    if self.flag_create_dataframes:
        # Two steps parsings: row count + row fill
        n=0
        for elem in root.iter():
          try:
            if( elem.tag ):
              if ( elem.tag == 'node' ):
                n += 1
          except:
            error(3,"Error parsing XML tree. Exception: "+str(sys.exc_info()))

        self.vprint( "Detected "+str(n)+" nodes" );
        self.nodes = pd.DataFrame(index=np.arange(n), columns=['id','x','y','type'] )
        
    n=0
    for elem in root.iter():
      try:
        if( elem.tag ):
          if ( elem.tag == 'node' ):
            if self.flag_create_dataframes:
                self.nodes.loc[n]=[elem.attrib['id'],elem.attrib['x'],elem.attrib['y'],elem.attrib['type']]
                n += 1
            self.geo_nodes[elem.attrib['id']]={}
            self.geo_nodes[elem.attrib['id']]['pt'] = TazPt(x=float(elem.attrib['x']),y=float(elem.attrib['y']))
      except:
        error(3,"Error parsing XML tree. Exception: "+str(sys.exc_info()))

    # if self.flag_create_dataframes:
    #    print( self.geo_nodes )
    self.vprint( "End parsing nodes file" );


  # -----------------------------------------------
  def loadEdges(self,file):
    self.vprint( "Start parsing edges file" );
    root = {}
    try:
      tree=etree.parse(file)
      root=tree.getroot()
    except:
      error(2,"Parsing "+file+" file. Exception: "+str(sys.exc_info()) )

    if self.flag_create_dataframes:
        # Two steps parsings: row count + row fill
        n=0
        for elem in root.iter():
          try:
            if( elem.tag ):
              if ( elem.tag == 'edge' ):
                n += 1
          except:
            error(3,"Error parsing XML tree. Exception: "+str(sys.exc_info()))

        self.vprint( "Detected "+str(n)+" edges" );
        self.edges = pd.DataFrame(index=np.arange(n),
                              columns=['id','from','to','prio','type','numLanes','speed'] )
    n=0
    for elem in root.iter():
      try:
        if( elem.tag ):
          if ( elem.tag == 'edge' ):
            if self.flag_create_dataframes:
                self.edges.loc[n]=[elem.attrib['id'],elem.attrib['from'],elem.attrib['to'],
                                   elem.attrib['priority'],elem.attrib['type'],
                                   elem.attrib['numLanes'],elem.attrib['speed']]
                n += 1
            edge_name = ''
            if 'name' in elem.attrib:
                edge_name = elem.attrib['name']
            self.edges[elem.attrib['id']] ={ 'from':elem.attrib['from'],
                                             'to': elem.attrib['to'],
                                             'prio': elem.attrib['priority'],
                                             'type': elem.attrib['type'],
                                             'numLanes': elem.attrib['numLanes'],
                                             'speed': elem.attrib['speed'],
                                             'name': edge_name
                                           }
            
      except:
        error(3,"Error parsing XML tree. Exception: "+str(sys.exc_info()))

    # if self.flag_create_dataframes:
    #    print( self.edges )

    self.vprint( "End parsing edges file" );

  # -----------------------------------------------
  def loadNet(self,file):
    self.vprint( "Start parsing net file" );
    self.vprint( "End parsing net file" );

  # -----------------------------------------------
  def loadMUTaz(self,file):
    self.vprint( "Start parsing MuTaz file" );
    root = {}
    try:
      tree = etree.parse(file)
      root=tree.getroot()
    except:
      error(2,"Parsing "+file+" file. Exception: "+str(sys.exc_info()) )

    if self.flag_create_dataframes:
        # Two steps parsings: row count + row fill
        n=0
        for elem in root.iter():
          # etree.dump(elem)
          try:
            if( elem.tag ):
              if ( elem.tag == 'vertex' ):
                n += 1
          except:
            error(3,"Error parsing XML tree. Exception: "+str(sys.exc_info()))

        self.vprint( "Detected "+str(n)+" mutazs" );
        self.mutazs = pd.DataFrame(index=np.arange(n), columns=['mutaz','vertex','x','y'] )
        
    n=0
    curr_mutaz = ''
    for elem in root.iter():
      try:
        if( elem.tag ):
          if( elem.tag == 'mutaz' ):
            # print( elem.attrib )
            curr_mutaz = elem.attrib['id']
            self.geometry[curr_mutaz] = {'contained_nodes':{},
                                         'contained_edges':{},
                                         'taz_vertices':[],
                                         'taz_edges':[],
                                         'taz_poligon': None }
          if ( elem.tag == 'vertex' ):
            # print( elem.attrib )
            if self.flag_create_dataframes:
                self.mutazs.loc[n]=[curr_mutaz,elem.attrib['id'],elem.attrib['x'],elem.attrib['y']]
                n += 1
            self.geometry[curr_mutaz]['taz_vertices'].append( TazPt(x=float(elem.attrib['x']),y=float(elem.attrib['y'])))
      except:
        error(3,"Error parsing XML tree. Exception: "+str(sys.exc_info()))
    
    # Geometry calculus
    for taz in self.geometry:
        for n in range(0,len(self.geometry[taz]['taz_vertices'])-1):
            self.geometry[taz]['taz_edges'].append( TazEdge(a=self.geometry[taz]['taz_vertices'][n], b=self.geometry[taz]['taz_vertices'][n+1]) )
        self.geometry[taz]['taz_edges'].append( TazEdge(a=self.geometry[taz]['taz_vertices'][n+1], b=self.geometry[taz]['taz_vertices'][0]) )
        self.geometry[taz]['taz_poligon'] = TazPolygon(name=taz, edges=tuple(self.geometry[taz]['taz_edges']) )
    # self.vprint( self.geometry[taz]['taz_poligon'] )
    self.vprint( "End parsing MuTaz file" );

  # -----------------------------------------------------------
  def loadData(self):
    if not os.path.isfile(self.opts['in_net']):
      error(1,'Network file doesnt exist')
    if not os.path.isfile(self.opts['in_nodes']):
      error(2,'Nodes file doesnt exist')
    if not os.path.isfile(self.opts['in_edges']):
      error(3,'Edges file doesnt exist')
    if not os.path.isfile(self.opts['in_mutaz']):
      error(3,'Taz-XML file doesnt exist')

    self.loadNodes(self.opts['in_nodes'])
    self.loadEdges(self.opts['in_edges'])
    self.loadNet(self.opts['in_net'])
    self.loadMUTaz(self.opts['in_mutaz'])

  def calculateTazs(self):
    for taz in self.geometry:
        poly = self.geometry[taz]['taz_poligon']
        self.vprint( "******** TAZ "+taz+" ********")
        
        # STEP 1: Check which nodes are contained inside the MUTAZ polygon
        self.vprint( " NODES CONTAINED:")
        for pt in self.geo_nodes:
            if taz_contains_pt(poly, self.geo_nodes[pt]['pt']):
                self.geometry[taz]['contained_nodes'][pt] = True
                # self.vprint( "--> "+pt)
                
        for pt in self.geometry[taz]['contained_nodes']:
            if self.geometry[taz]['contained_nodes'][pt]:
                self.vprint( "--> "+pt)
                
        # STEP 2: Check which edges have both nodes included inside the MUTAZ polygon
        self.vprint( " EDGES CONTAINED (FULL/PARTIAL):")
        for edge in self.edges:
            if( self.edges[edge]['from'] in self.geometry[taz]['contained_nodes'] ):
                if( self.edges[edge]['to'] in self.geometry[taz]['contained_nodes'] ):
                    self.geometry[taz]['contained_edges'][edge] = True
                    self.vprint( "--> FULL: "+edge)
                else:
                    self.vprint( "--> PARTIAL (not included): "+edge)

  def dumpTazFile(self):
        print( '<tazs>')
        for taz in self.geometry:
            print( '  <taz id="'+taz+'">')
            weight=1/len(self.geometry[taz]['contained_edges'])
            for edge in self.geometry[taz]['contained_edges']:
                print( '    <!-- name="'+self.edges[edge]['name']+'" -->')
                print( '    <tazSource id="'+edge+'" weight="'+str(weight)+'">')
                print( '    <tazSink id="'+edge+'" weight="'+str(weight)+'">')
            print( '  </taz>')
        print( '</tazs>')


In [28]:
CITY = "alcalahenares"

opts={}
opts['net_path']="/Users/alvaro/Desktop/workspace/mutraff/uah-gist-mutraff-bastra/tools/TAZ_CALCULATOR"
opts['in_net']=opts['net_path']+'/'+CITY+".net.xml"
opts['in_nodes']=opts['net_path']+'/'+CITY+".nod.xml"
opts['in_edges']=opts['net_path']+'/'+CITY+".edg.xml"
opts['in_mutaz']=opts['net_path']+'/'+CITY+"2.mutaz.xml"
opts['in_mutaz']=opts['net_path']+'/alcalahenares.minicentro.mutaz.xml'
opts['verbose']=True
tazcalc = MuTazCalculator(opts)
tazcalc.loadData()

Start parsing nodes file
End parsing nodes file
Start parsing edges file
End parsing edges file
Start parsing net file
End parsing net file
Start parsing MuTaz file
End parsing MuTaz file


In [29]:
tazcalc.calculateTazs()

******** TAZ minicentro ********
 NODES CONTAINED:
--> 1472473123
--> 258721361
--> 765364238
--> 1472473179
--> 1472473094
--> 259091763
--> 259091708
--> 258720885
--> 913994209
--> 259091742
--> 258720886
--> 1472473064
--> 258577807
--> 4396492442
--> 258719532
--> 765364237
--> 3086181214
--> 252862471
--> 4396492444
--> 258578505
--> 258721211
--> 258577956
--> 258720887
--> 1472473020
--> 1472472993
--> 4396492439
--> 259092612
--> 259091740
--> 258720888
--> 259091738
--> 258577955
--> 258720884
--> 1472473012
 EDGES CONTAINED (FULL/PARTIAL):
--> FULL: -23352493#1
--> FULL: 78579994
--> FULL: -61238145#0
--> PARTIAL (not included): 23352493#3
--> FULL: 23870257
--> FULL: 23858626#0
--> FULL: 61238148#2
--> PARTIAL (not included): 23870217#0
--> FULL: 61238148#3
--> PARTIAL (not included): 404635470#0
--> FULL: -408875679#0
--> FULL: 23858626#1
--> FULL: 61238145#1
--> FULL: 23858590#1
--> FULL: 23352493#2
--> FULL: -23352493#2
--> FULL: 61238148#1
--> FULL: 61238148#0
--> PARTI

In [26]:
tazcalc.dumpTazFile()

<tazs>
  <taz id="minicentro">
    <!-- name="Plaza de Cervantes" -->
    <tazSource id="23858625#3" weight="0.019230769230769232">
    <tazSink id="23858625#3" weight="0.019230769230769232">
    <!-- name="Plaza de San Diego" -->
    <tazSource id="-61238148#1" weight="0.019230769230769232">
    <tazSink id="-61238148#1" weight="0.019230769230769232">
    <!-- name="Calle Beatas" -->
    <tazSource id="23904464" weight="0.019230769230769232">
    <tazSink id="23904464" weight="0.019230769230769232">
    <!-- name="Plaza Cervantes" -->
    <tazSource id="23858626#0" weight="0.019230769230769232">
    <tazSink id="23858626#0" weight="0.019230769230769232">
    <!-- name="Plaza de Cervantes" -->
    <tazSource id="23858625#0" weight="0.019230769230769232">
    <tazSink id="23858625#0" weight="0.019230769230769232">
    <!-- name="Calle de San Pedro y San Pablo" -->
    <tazSource id="-23904598" weight="0.019230769230769232">
    <tazSink id="-23904598" weight="0.019230769230769232">
    